In [17]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time 
    #json library
import json
    #Analyst tools
import sys

sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query, run_read_prod_query

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from procurement_lib import GoogleSheet

In [18]:
df_gtin = GoogleSheet("1TBAwmxAoqg6FX79d7wLu3DVtIiOb6gRfkN5fMA_tLNc")
df_gtin = df_gtin.get_as_dataframe('Hoja 1')

In [19]:
df = pd.read_csv("info_assai_atacado.csv")
df['quotation_date'] = pd.to_datetime(df['quotation_date'])
df[['gtin']] = df[['gtin']].astype('float')

In [20]:
df2 = pd.merge(df, df_gtin, left_on=['gtin','site_code'], right_on=['GTIN','city'], how='left')
df2 = df2.drop(['city', 'GTIN'], axis=1)
df2 = df2.dropna().reset_index(drop=True)

df2['lifetime'] = 8

In [21]:
# Ensure dataframe is sorted by 'quotation_date'
df2 = df2.sort_values(by='quotation_date')

# Generate the required rows for missing dates
new_rows = []

for (competitor, source_id), group in df2.groupby(['competitor_name', 'source_id']):
    group = group.sort_values(by='quotation_date')
    last_known_price = None
    last_known_date = None
    lifetime = 8
    
    for current_index in range(len(group)):
        current_date = group.iloc[current_index]['quotation_date']
        price = group.iloc[current_index]['price']
        
        # If this is not the first iteration, fill in missing dates
        if last_known_date is not None:
            days_diff = (current_date - last_known_date).days
            if days_diff > 1:
                for j in range(1, min(days_diff, lifetime + 1)):
                    new_date = last_known_date + timedelta(days=j)
                    new_row = {
                        'site_code': group.iloc[current_index]['site_code'],
                        'quotation_date': new_date,
                        'dim_quotation_date': int(new_date.strftime('%Y%m%d')),
                        'competitor_name': competitor,
                        'source_id': source_id,
                        'product_name': group.iloc[current_index]['product_name'],
                        'price': last_known_price,
                        'lifetime': lifetime - j
                    }
                    new_rows.append(new_row)
                    
                    # Stop if we reach a new datapoint date
                    if new_date + timedelta(days=1) == current_date:
                        break
        
        # Update the last known values and reset lifetime
        last_known_price = price
        last_known_date = current_date
        lifetime = 8  # Reset lifetime

    # After processing all known dates for the group, continue generating rows until lifetime reaches 0
    while lifetime > 0:
        last_known_date += timedelta(days=1)
        new_row = {
            'site_code': group.iloc[-1]['site_code'],
            'quotation_date': last_known_date,
            'dim_quotation_date': int(last_known_date.strftime('%Y%m%d')),
            'competitor_name': competitor,
            'source_id': source_id,
            'product_name': group.iloc[-1]['product_name'],
            'price': last_known_price,
            'lifetime': lifetime - 1
        }
        new_rows.append(new_row)
        lifetime -= 1

# Append new rows to the dataframe
df2 = df2.append(new_rows, ignore_index=True)

# Sort the final dataframe
df2 = df2.sort_values(by=['competitor_name', 'source_id', 'quotation_date'])
df2['replica'] = df2['lifetime'] == 8

In [22]:
df2[['site_code', 'quotation_date', 'competitor_name', 'price',
       'source_id', 'product_name']].to_csv("all_infoprice_info_no_out_by_store.csv", index = False)

In [23]:
df_bench = pd.read_csv("all_infoprice_info_no_out_by_store.csv")

# Convert the quotation_date column to datetime format
df_bench['quotation_date'] = pd.to_datetime(df_bench['quotation_date'])
df_bench['product_name'] = df_bench['site_code'] + " || " + df_bench['product_name']

In [25]:
query = """
WITH RECURSIVE calendar(calendar_date) AS (
  SELECT DATE_TRUNC('day', DATE(GETDATE()) - INTERVAL '700 day')
  UNION ALL
  SELECT calendar_date + INTERVAL '1 day'
  FROM calendar
  WHERE calendar_date BETWEEN DATE_TRUNC('day', DATE(GETDATE()) - INTERVAL '700 day') AND DATE(GETDATE() - 1) 
),

info AS (
SELECT
    DATE(coalesce(prices.last_modified_at, prices.created_at)) as created_at,
    pp.frida_id as source_id,
    MIN(coalesce(tiers.tax_price, prices.tax_price)) as price,
    MIN(coalesce(tiers.sale_price, prices.sale_price)) as net_price

FROM postgres_growth."growth_pricing.prices_history" prices
LEFT JOIN postgres_growth."growth_pricing.price_tiers_history" tiers ON prices.id = tiers.price_history_id
LEFT JOIN postgres_growth."growth_pricing.skus" skus ON prices.sku_id = skus.id
LEFT JOIN postgres_main_co."purchase_orders.products" p ON skus.sku_id = p.frida_id
LEFT JOIN postgres_main_co."purchase_orders.products" pp ON COALESCE(p.parent_id, p.id) = pp.id

WHERE DATE(prices.created_at) >= DATE_TRUNC('day', DATE(GETDATE()) - INTERVAL '700 day')
 AND p.region_code IN ('SPO','BHZ','CWB','VCP')
 AND p.deleted_at IS NULL
 AND prices.created_by NOT ILIKE '%CATALOG%'
 AND pp.product_category_id IN (5,6,7,8,9,10,13,18) -- 1 ES FRUVER
 AND pp.frida_id IN {skus}
GROUP BY 1,2--,3,4
),

done AS (
SELECT
  DATE(c.calendar_date) AS quotation_date,
  --s.region,
  --s.parent_product_name,
  (s.source_id)::int as source_id,
  LAG(i.price IGNORE NULLS) OVER (PARTITION BY s.source_id ORDER BY c.calendar_date)::FLOAT AS p_price_tool,
  LAG(i.net_price IGNORE NULLS) OVER (PARTITION BY s.source_id ORDER BY c.calendar_date)::FLOAT AS net_price_tool


FROM calendar c
CROSS JOIN (SELECT DISTINCT source_id FROM info) s
LEFT JOIN info i ON c.calendar_date = i.created_at-1 AND s.source_id = i.source_id
)

SELECT *
FROM done
WHERE net_price_tool IS NOT NULL
 AND quotation_date >= '2024-02-10'
 AND quotation_date <= '2024-05-22'
""".format(skus=tuple(df_bench.source_id.unique()))
df = run_read_prod_query(query)  
df['quotation_date'] = pd.to_datetime(df['quotation_date'])

In [27]:
df = pd.merge(df_bench, df, left_on=['source_id','quotation_date'], right_on=['source_id','quotation_date'], how='inner')

In [28]:
query = """
SELECT
    s.identifier_value AS site_code,
    DATE(fs.order_submitted_date) AS quotation_date,
    sup.source_id,
    (SUM(fs.gmv_pxq_local)/4.75)::FLOAT AS gmv_usd,
    gmv_usd/SUM(gmv_usd) OVER (PARTITION BY s.identifier_value)::FLOAT AS gmv_mix,
    AVG(COALESCE(inventory_p_fin,cogs_p_day))::FLOAT AS costo

FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
INNER JOIN dpr_shared.dim_stock_unit        su  ON su.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_stock_unit        sup  ON nvl(nullif(su.source_parent_id,0),su.source_id) = sup.source_id
LEFT JOIN dpr_cross_business.fact_cross_business_insights m ON m.dim_stock_unit = sup.stock_unit_id AND m.dim_date = fs.dim_submitted_date

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND cat.super_category = 'Multicategoría'
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND fs.gmv_pxq_local > 0
    AND s.identifier_value IN ('SPO','CWB','VCP','BHZ')
    AND DATE(fs.order_submitted_date) >= '2024-02-10'
    AND DATE(fs.order_submitted_date) <= '2024-05-22'
GROUP BY 1,2,3
HAVING costo > 0
"""
df_gmv = run_read_dwd_query(query)
df_gmv['quotation_date'] = pd.to_datetime(df_gmv['quotation_date'])

In [30]:
df_assai = pd.merge(df_gmv, df.loc[df.competitor_name=='Assai Atacadista'], left_on=['site_code','source_id','quotation_date'], right_on=['site_code','source_id','quotation_date'], how='left')
df_atacado = pd.merge(df_gmv, df.loc[df.competitor_name=='Atacadão'], left_on=['site_code','source_id','quotation_date'], right_on=['site_code','source_id','quotation_date'], how='left')

In [31]:
df_assai['gpi'] = df_assai.p_price_tool/df_assai.price
df_assai['npi'] = df_assai.net_price_tool/df_assai.price

df_assai['mg'] = 1-(df_assai.costo/df_assai.p_price_tool)
df_assai['nmg'] = 1-(df_assai.costo/df_assai.net_price_tool)

df_assai['comp_mg'] = 1-(df_assai.costo/df_assai.price)

In [32]:
df_atacado['gpi'] = df_atacado.p_price_tool/df_atacado.price
df_atacado['npi'] = df_atacado.net_price_tool/df_atacado.price

df_atacado['mg'] = 1-(df_atacado.costo/df_atacado.p_price_tool)
df_atacado['nmg'] = 1-(df_atacado.costo/df_atacado.net_price_tool)

df_atacado['comp_mg'] = 1-(df_atacado.costo/df_atacado.price)

In [35]:
# Define a function to apply the custom aggregation
def custom_ventas(group, column):
    return (100.00 * group[column] * group['gmv_mix']).sum()/group['gmv_mix'].sum()
merge_df2 = df_assai.dropna(how='any')
# Group by 'city' and 'year_week', aggregate multiple columns with custom aggregation
df_final_assai = merge_df2.groupby(['site_code']).apply(lambda group: pd.Series({
    'gmv_mix': 100.00*group['gmv_mix'].sum(),  # Compute sum of gmv_mix directly
    'skus': group['source_id'].nunique(),
    'mg': custom_ventas(group, 'mg'),
    'nmg': custom_ventas(group, 'nmg'),
    'comp_mg': custom_ventas(group, 'comp_mg'),
    'gpi': custom_ventas(group, 'gpi'),
    'npi': custom_ventas(group, 'npi')
})).reset_index()
print('INFOPRICE -ASSAI INFO:')
df_final_assai

INFOPRICE -ASSAI INFO:


,site_code,gmv_mix,skus,mg,nmg,comp_mg,gpi,npi
0,BHZ,41.298247,218.0,8.722071,4.279626,5.566946,103.951868,99.420635
1,CWB,17.643829,110.0,8.982334,5.435905,4.757701,104.805434,100.968994
2,SPO,58.192791,428.0,9.347103,6.644135,11.960858,97.216553,94.543421
3,VCP,46.153851,253.0,8.601947,5.315976,11.483292,96.941102,93.659933


In [37]:
# Define a function to apply the custom aggregation
def custom_ventas(group, column):
    return (100.00 * group[column] * group['gmv_mix']).sum()/group['gmv_mix'].sum()
merge_df2 = df_atacado.dropna(how='any')
# Group by 'city' and 'year_week', aggregate multiple columns with custom aggregation
df_final_atacado = merge_df2.groupby(['site_code']).apply(lambda group: pd.Series({
    'gmv_mix': 100.00*group['gmv_mix'].sum(),  # Compute sum of gmv_mix directly
    'skus': group['source_id'].nunique(),
    'mg': custom_ventas(group, 'mg'),
    'nmg': custom_ventas(group, 'nmg'),
    'comp_mg': custom_ventas(group, 'comp_mg'),
    'gpi': custom_ventas(group, 'gpi'),
    'npi': custom_ventas(group, 'npi')
})).reset_index()
print('INFOPRICE -ATACADAO INFO:')
df_final_atacado

INFOPRICE -ATACADAO INFO:


,site_code,gmv_mix,skus,mg,nmg,comp_mg,gpi,npi
0,BHZ,37.445107,184.0,8.614564,3.492398,9.504738,99.273602,94.320159
1,CWB,23.828168,170.0,9.194811,5.891820,6.548380,103.697914,100.451990
2,SPO,54.885213,415.0,9.652923,6.782515,4.529234,105.876297,102.693496
3,VCP,37.923383,243.0,9.378060,6.025572,13.577839,95.478007,92.148891
